In [1]:
pip install gspread oauth2client pandas pyodbc

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pyodbc
from datetime import datetime

# Google Sheets setup
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    "clean-bindery-467706-r4-83555049562f.json", scope
)
client = gspread.authorize(creds)
sheet = client.open("Customer deatails (Responses)").sheet1
records = sheet.get_all_records()

# SQL Server connection
conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=SAMEER\SQLEXPRESS;"
    r"DATABASE=version1;"
    r"Trusted_Connection=yes;"
)
cursor = conn.cursor()

# Ensure table exists
cursor.execute("""
IF NOT EXISTS (
    SELECT * FROM sysobjects WHERE name='FormResponses' AND xtype='U'
)
BEGIN
    CREATE TABLE FormResponses (
        Timestamp DATETIME,
        CustomerType NVARCHAR(100),
        CustomerName NVARCHAR(100),
        Email NVARCHAR(100),
        Phone BIGINT,
        Item NVARCHAR(100),
        ProductOption INT,
        Comments NVARCHAR(500)
    )
END
""")
conn.commit()

# Insert only new records
inserted_count = 0
for record in records:
    try:
        timestamp = datetime.strptime(record['Timestamp'], "%d/%m/%Y %H:%M:%S")
        customer_type = record['Are you a new or existing customer?']
        name = record['Your name']
        email = record['E-mail']
        phone = int(record['Phone number'])
        item = record['What is the item you would like to order?']
        product_option = int(record['Product options'])
        comments = record['Questions and comments']

        # Prevent duplicates
        cursor.execute("SELECT COUNT(*) FROM FormResponses WHERE Timestamp = ?", (timestamp,))
        if cursor.fetchone()[0] == 0:
            cursor.execute("""
                INSERT INTO FormResponses (
                    Timestamp, CustomerType, CustomerName, Email,
                    Phone, Item, ProductOption, Comments
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (timestamp, customer_type, name, email, phone, item, product_option, comments))
            inserted_count += 1
    except Exception as e:
        print(f"❌ Error with record [{record.get('Your name', 'Unknown')}]: {e}")

conn.commit()
conn.close()
print(f"✅ Inserted {inserted_count} new records into SQL Server.")
